In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from survival import DecayMIRModel
from survival import MIRModel
from survival.age_group_extension import AgeSurvivalModel
from survival.three_parameter_log_form import LogFormMIRModel

## INPUT DATA

### [GBD 2019 MIRs with GBD 2019 background mortality]

In [ ]:
#df = pd.read_csv('/home/j/temp/fed1/2019_mirs_with_background_mortality_ALL_only.csv')

### [GBD 2020 v 89 MIRs with GBD 2020 background mortality]

In [20]:
df = pd.read_csv('/home/j/temp/fed1/2020_mirs_with_background_mortality_ALL_only.csv')

In [21]:
#PROCESS INPUTS
#remove NAs and MIR >1
df = df.dropna()
df = df[df['mi_ratio']<=1]
df = df[df['age_group_id']!=22]
df = df[df['age_group_id']!=28]
df = df[df['age_group_id']!=2]
df = df[df['age_group_id']!=3]
df = df[df['age_group_id']!=4]
df = df[df['age_group_id']!=5]

df.head()

,Unnamed: 0,age_group_id,sex_id,year_id,location_id,acause,mi_ratio,cause_id,measure_id,metric_id,age_group_name,cause_name,expected,location_name,location_type,measure_name,metric_name,sex,background_mortality
0,1,1,1,1990,6,neo_leukemia_ll_acute,0.736437,294,1,3,Under 5,All causes,False,China,admin0,Deaths,Rate,Male,0.010942
1,2,1,1,1990,7,neo_leukemia_ll_acute,0.747966,294,1,3,Under 5,All causes,False,Democratic People's Republic of Korea,admin0,Deaths,Rate,Male,0.009047
2,3,1,1,1990,8,neo_leukemia_ll_acute,0.254112,294,1,3,Under 5,All causes,False,Taiwan (Province of China),admin0,Deaths,Rate,Male,0.001818
3,4,1,1,1990,10,neo_leukemia_ll_acute,0.951282,294,1,3,Under 5,All causes,False,Cambodia,admin0,Deaths,Rate,Male,0.029895
4,5,1,1,1990,11,neo_leukemia_ll_acute,0.890948,294,1,3,Under 5,All causes,False,Indonesia,admin0,Deaths,Rate,Male,0.018975


In [22]:
# Subset to the pediatric age groups and test on a single year (2019)
df = df[df['age_group_id']<9]
df = df[df['year_id']==2020]

## I. Base MIR -> Survival Model:

#### [M/I = (1-P_s^n)(P_c)/(P_c+P_o)]

In [23]:
#PARAMETERS
num_years = 5
disease_pd = 10

#RUN BASE MODEL
model = MIRModel(df['mi_ratio'],
                 df['background_mortality'],
                 disease_period=disease_pd)
model.compute_excess_mortality()
survival_rate = model.get_survival_rate(num_years=num_years)

# SAVE IN ORIGINAL DATA FRAME
df['excess_mortality_og_mir_model'] = model.excess_mortality
df['abs_survival_rate_og_mir_model'] = survival_rate['abs']
df['rel_survival_rate_og_mir_model'] = survival_rate['rel']

## IV. Three parameter decay model:

#### [log(P_c) = b*log(1+exp(-(x-a)))+c]

In [24]:
# merge in the fitted parameters
parameters = pd.read_csv('/homes/fed1/Survival_Franny/three_parameter_log_form_values_SEER_6_9_2021_with_acause.csv')
df = df.merge(parameters, how="left", on=['acause'])

In [25]:
num_years = 5

# RUN THREE PARAMETER LOG MODEL
three_param_model = LogFormMIRModel(df['mi_ratio'],
                                    df['background_mortality'],
                                    df['a'],
                                    df['b'])
three_param_model.compute_third_parameter()
survival_rate = three_param_model.get_survival_rate(num_years=num_years)

# SAVE IN ORIGINAL DATA FRAME
df['predicted_c'] = three_param_model.c
df['abs_survival_rate_three_param_model'] = survival_rate['abs']
df['rel_survival_rate_three_param_model'] = survival_rate['rel']

## Comparison to Current GBD Methods

In [26]:
df['rel_survival_old_gbd_model'] = 1-df['mi_ratio']

## Save results

In [27]:
df.to_csv('/home/j/temp/fed1/comparing_ALL_survival_methods_2020_inputs_6_15_2021.csv')

In [28]:
df.head()

,Unnamed: 0_x,age_group_id,sex_id,year_id,location_id,acause,mi_ratio,cause_id,measure_id,metric_id,...,rel_survival_rate_og_mir_model,Unnamed: 0_y,cause_name_y,a,b,c,predicted_c,abs_survival_rate_three_param_model,rel_survival_rate_three_param_model,rel_survival_old_gbd_model
0,7323,1,1,2020,6,neo_leukemia_ll_acute,0.118296,294,1,3,...,0.938480,89,Acute Lymphocytic Leukemia,10.599583,0.39187,-5.714212,-6.790519,0.884909,0.891562,0.881704
1,7324,1,1,2020,7,neo_leukemia_ll_acute,0.448821,294,1,3,...,0.739309,89,Acute Lymphocytic Leukemia,10.599583,0.39187,-5.714212,-5.174394,0.572557,0.578430,0.551179
2,7325,1,1,2020,8,neo_leukemia_ll_acute,0.054866,294,1,3,...,0.972052,89,Acute Lymphocytic Leukemia,10.599583,0.39187,-5.714212,-7.602913,0.946125,0.950062,0.945134
3,7326,1,1,2020,10,neo_leukemia_ll_acute,0.730656,294,1,3,...,0.496069,89,Acute Lymphocytic Leukemia,10.599583,0.39187,-5.714212,-4.274729,0.283714,0.294112,0.269344
4,7327,1,1,2020,11,neo_leukemia_ll_acute,0.676669,294,1,3,...,0.552851,89,Acute Lymphocytic Leukemia,10.599583,0.39187,-5.714212,-4.455017,0.341089,0.350959,0.323331
